In [1]:
# from helper_functions.tourney_sim import simulate_tournament
import os
import numpy as np
import pickle
from tqdm import tqdm
import pandas as pd
pd.set_option('display.max_columns', 90)

C:\Users\mdona\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\mdona\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
real_winners = ['Michigan St.','Duquesne','Creighton', 'Arizona', 'North Carolina','Illinois','Oregon', 'Dayton',
                'Texas','Oakland','Gonzaga','Iowa St.','Tennessee', 'NC State', 'Washington St.','Kansas']
real_losers = ['Mississippi St.','BYU', 'Akron', 'Long Beach St.', 'Wagner', 'Morehead St.','South Carolina','Nevada',
               'Colorado St.', 'Kentucky','McNeese','South Dakota St.',"Saint Peter's", 'Texas Tech','Drake','Samford']

In [3]:
game_ids = [9, 4, 29, 15, 8, 5, 28, 14, 30, 21, 26, 7, 31, 20,6,27]

In [4]:
lines = {'Marquette':-192,
         'Purdue':-625,
         'Duke':-285,
         'Baylor':-198,
         'Grand Canyon':195,
         'UConn':-1050,
         'Houston':-485,
         'San Diego St.':-230}

In [5]:
probs={'Marquette':.798,
       'Purdue':.687,
       'Duke':.640,
       'Baylor':.607,
       'Grand Canyon':.589,
       'UConn':.885,
       'Houston':.586,
       'San Diego St.':.615}

In [6]:
def implied_prob(line):
    if line < 0:
        ip = (-1*line)/(-1*line+100)
    else:
        ip=100/(line+100)
    return ip

In [7]:
def reverse_prob(p):
    if p <= .5:
        line = (100/p)-100
    else:
        line = (100*p)/(p-1)
    return line


In [8]:
def payout(line, bet):
    winnings=0
    if line < 0:
        winnings = bet/(-line/100)
    else:
        winnings = bet * (line/100)
    return (bet + winnings)

In [9]:
def combine_lines(lines=[]):
    probs=[]
    for l in lines:
        probs.append(implied_prob(l))
    total_prob=1
    for p in probs:
        total_prob*=p
    return reverse_prob(total_prob)


In [10]:
def find_ones_indexes(binary_tuple):
    indexes = []
    for index, bit in enumerate(binary_tuple):
        if bit == 1:
            indexes.append(index+1)
    return indexes

In [11]:
from itertools import combinations

In [12]:
n=8

In [13]:
binary_tuples = [tuple(int(bit) for bit in bin(i)[2:].zfill(n)) for i in range(2**n)]

In [14]:
binary_tuples

[(0, 0, 0, 0, 0, 0, 0, 0),
 (0, 0, 0, 0, 0, 0, 0, 1),
 (0, 0, 0, 0, 0, 0, 1, 0),
 (0, 0, 0, 0, 0, 0, 1, 1),
 (0, 0, 0, 0, 0, 1, 0, 0),
 (0, 0, 0, 0, 0, 1, 0, 1),
 (0, 0, 0, 0, 0, 1, 1, 0),
 (0, 0, 0, 0, 0, 1, 1, 1),
 (0, 0, 0, 0, 1, 0, 0, 0),
 (0, 0, 0, 0, 1, 0, 0, 1),
 (0, 0, 0, 0, 1, 0, 1, 0),
 (0, 0, 0, 0, 1, 0, 1, 1),
 (0, 0, 0, 0, 1, 1, 0, 0),
 (0, 0, 0, 0, 1, 1, 0, 1),
 (0, 0, 0, 0, 1, 1, 1, 0),
 (0, 0, 0, 0, 1, 1, 1, 1),
 (0, 0, 0, 1, 0, 0, 0, 0),
 (0, 0, 0, 1, 0, 0, 0, 1),
 (0, 0, 0, 1, 0, 0, 1, 0),
 (0, 0, 0, 1, 0, 0, 1, 1),
 (0, 0, 0, 1, 0, 1, 0, 0),
 (0, 0, 0, 1, 0, 1, 0, 1),
 (0, 0, 0, 1, 0, 1, 1, 0),
 (0, 0, 0, 1, 0, 1, 1, 1),
 (0, 0, 0, 1, 1, 0, 0, 0),
 (0, 0, 0, 1, 1, 0, 0, 1),
 (0, 0, 0, 1, 1, 0, 1, 0),
 (0, 0, 0, 1, 1, 0, 1, 1),
 (0, 0, 0, 1, 1, 1, 0, 0),
 (0, 0, 0, 1, 1, 1, 0, 1),
 (0, 0, 0, 1, 1, 1, 1, 0),
 (0, 0, 0, 1, 1, 1, 1, 1),
 (0, 0, 1, 0, 0, 0, 0, 0),
 (0, 0, 1, 0, 0, 0, 0, 1),
 (0, 0, 1, 0, 0, 0, 1, 0),
 (0, 0, 1, 0, 0, 0, 1, 1),
 (0, 0, 1, 0, 0, 1, 0, 0),
 

In [15]:
rr_possibilities = binary_tuples.copy()

In [17]:
probs.keys()

dict_keys(['Marquette', 'Purdue', 'Duke', 'Baylor', 'Grand Canyon', 'UConn', 'Houston', 'San Diego St.'])

In [18]:
best_rr = []
moneys = []
all_df = pd.DataFrame()

for i in tqdm(range(10000)):
    current_success = (np.random.choice([1,0],p=[probs['Marquette'], 1-probs['Marquette']]),
                       np.random.choice([1,0],p=[probs['Purdue'], 1-probs['Purdue']]),
                       np.random.choice([1,0],p=[probs['Duke'], 1-probs['Duke']]),
                       np.random.choice([1,0],p=[probs['Baylor'], 1-probs['Baylor']]),
                       np.random.choice([1,0],p=[probs['Grand Canyon'], 1-probs['Grand Canyon']]),
                       np.random.choice([1,0],p=[probs['UConn'], 1-probs['UConn']]),
                       np.random.choice([1,0],p=[probs['Houston'], 1-probs['Houston']]),
                       np.random.choice([1,0],p=[probs['San Diego St.'], 1-probs['San Diego St.']]))
    success_dict = dict(zip(lines.keys(), current_success))
    rr_moneys = []
    for c in rr_possibilities:
        cost = 0
        current_payout = 0
        current_rrs = find_ones_indexes(c)
        for i in current_rrs:
            current_combinations =list(combinations(lines.keys(),i))
            for combo in current_combinations:
                line = combine_lines([lines[x] for x in combo])
                cost += 0.1
                win = True
                for choice in combo:
                    if success_dict[choice] == 0:
                        win=False
                if win:
                    current_payout += payout(line,0.1)
        rr_moneys.append(current_payout-cost)
    df = pd.DataFrame(zip(rr_possibilities,rr_moneys)).sort_values(by=1,ascending=False)
    df['iter']=i
    all_df=pd.concat([all_df, df])
    best_rr.append(df[0].iloc[0])
    moneys.append(df[1].iloc[0])


  6%|▌         | 602/10000 [00:42<10:57, 14.29it/s]


KeyboardInterrupt: 

In [24]:
all_df

,0,1,iter
0,"(0, 0, 0, 0, 0, 0, 0, 0)",0.000000,1
128,"(1, 0, 0, 0, 0, 0, 0, 0)",-0.092888,1
1,"(0, 0, 0, 0, 0, 0, 0, 1)",-0.100000,1
129,"(1, 0, 0, 0, 0, 0, 0, 1)",-0.192888,1
2,"(0, 0, 0, 0, 0, 0, 1, 0)",-0.800000,1
...,...,...,...
253,"(1, 1, 1, 1, 1, 1, 0, 1)",-23.634749,1
126,"(0, 1, 1, 1, 1, 1, 1, 0)",-23.916975,1
127,"(0, 1, 1, 1, 1, 1, 1, 1)",-24.016975,1
254,"(1, 1, 1, 1, 1, 1, 1, 0)",-24.334749,1


In [19]:
new_df = all_df.groupby(0)[1].mean().reset_index().sort_values(by=1).copy()
new_df.columns=['RR Set','Expected Return']

In [20]:
new_df['Expected Return'] *= 10

In [21]:
new_df

,RR Set,Expected Return
12,"(0, 0, 0, 0, 1, 1, 0, 0)",-0.630148
14,"(0, 0, 0, 0, 1, 1, 1, 0)",-0.621285
13,"(0, 0, 0, 0, 1, 1, 0, 1)",-0.597380
15,"(0, 0, 0, 0, 1, 1, 1, 1)",-0.588518
140,"(1, 0, 0, 0, 1, 1, 0, 0)",-0.508131
...,...,...
115,"(0, 1, 1, 1, 0, 0, 1, 1)",1.313310
240,"(1, 1, 1, 1, 0, 0, 0, 0)",1.393697
242,"(1, 1, 1, 1, 0, 0, 1, 0)",1.402560
241,"(1, 1, 1, 1, 0, 0, 0, 1)",1.426465


In [136]:
all_df.groupby(0)[1].mean().reset_index().sort_values(by=1)

,0,1
0,"(0, 0, 0, 0)",0.000000
1,"(0, 0, 0, 1)",0.071078
8,"(1, 0, 0, 0)",0.075383
9,"(1, 0, 0, 1)",0.146461
2,"(0, 0, 1, 0)",0.211975
4,"(0, 1, 0, 0)",0.222267
3,"(0, 0, 1, 1)",0.283053
10,"(1, 0, 1, 0)",0.287357
5,"(0, 1, 0, 1)",0.293345
12,"(1, 1, 0, 0)",0.297649


In [115]:
pd.DataFrame(zip(best_rr,moneys))[0].value_counts()

0
(0, 0, 0, 0, 0, 0, 0, 0)    500
(1, 1, 0, 0, 0, 0, 0, 0)    183
(1, 1, 1, 1, 1, 1, 1, 0)    153
(1, 0, 0, 0, 0, 0, 0, 0)     73
(1, 1, 1, 1, 1, 1, 1, 1)     46
(1, 1, 1, 1, 1, 1, 0, 0)     25
(1, 1, 1, 0, 0, 0, 0, 0)     15
Name: count, dtype: int64

In [116]:
pd.DataFrame(zip(best_rr,moneys))

,0,1
0,"(0, 0, 0, 0, 0, 0, 0, 0)",0.000000
1,"(0, 0, 0, 0, 0, 0, 0, 0)",0.000000
2,"(0, 0, 0, 0, 0, 0, 0, 0)",0.000000
3,"(0, 0, 0, 0, 0, 0, 0, 0)",0.000000
4,"(1, 1, 1, 1, 1, 1, 1, 0)",18.703515
...,...,...
990,"(1, 0, 0, 0, 0, 0, 0, 0)",0.014633
991,"(0, 0, 0, 0, 0, 0, 0, 0)",0.000000
992,"(0, 0, 0, 0, 0, 0, 0, 0)",0.000000
993,"(1, 1, 0, 0, 0, 0, 0, 0)",0.166429


In [90]:
pd.DataFrame(zip(rr_possibilities,rr_moneys))

,0,1
0,"(0, 0, 0, 0, 0, 0, 0, 0)",0.000000
1,"(0, 0, 0, 0, 0, 0, 0, 1)",2.087327
2,"(0, 0, 0, 0, 0, 0, 1, 0)",11.347418
3,"(0, 0, 0, 0, 0, 0, 1, 1)",13.434744
4,"(0, 0, 0, 0, 0, 1, 0, 0)",26.575640
...,...,...
251,"(1, 1, 1, 1, 1, 0, 1, 1)",92.795916
252,"(1, 1, 1, 1, 1, 1, 0, 0)",105.936811
253,"(1, 1, 1, 1, 1, 1, 0, 1)",108.024138
254,"(1, 1, 1, 1, 1, 1, 1, 0)",117.284229


In [85]:
current_combinations

[('Arizona',
  'Kansas',
  'North Carolina',
  'Iowa St.',
  'NC State',
  'Tennesee',
  'Illinois',
  'Creighton')]

In [70]:
rr_possibilities[0]

(0, 0, 0, 0, 0, 0, 0, 0)

In [80]:
abc=find_ones_indexes((0,0,1,0,1,1,0,1))

In [81]:
abc

[3, 5, 6, 8]

In [117]:
d2={'MSU':160,
    'Oregon':160,
    'GCU':200,
    'Oakland':210}

In [118]:
all_combs = list(combinations(d2.keys(),2))

In [119]:
all_combs

[('MSU', 'Oregon'),
 ('MSU', 'GCU'),
 ('MSU', 'Oakland'),
 ('Oregon', 'GCU'),
 ('Oregon', 'Oakland'),
 ('GCU', 'Oakland')]

In [121]:
for k in all_combs:
    L = combine_lines([d2[x] for x in k])
    print("{}: {}".format(k, L))

('MSU', 'Oregon'): 576.0
('MSU', 'GCU'): 680.0000000000001
('MSU', 'Oakland'): 706.0
('Oregon', 'GCU'): 680.0000000000001
('Oregon', 'Oakland'): 706.0
('GCU', 'Oakland'): 830.0000000000001
